### 라이브러리

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from dotenv import load_dotenv

### API 키

In [ ]:
load_dotenv

# API 키 설정
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")

In [ ]:
# PDF 로드 및 분할
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 200, 
                                               chunk_overlap = 20,
                                               )
docs = text_splitter.split_documents(pages)

# OpenAI 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              openai_api_key = OPENAI_API_KEY,
                              )

# FAISS 벡터 저장소
vectorstore = FAISS.from_documents(docs, 
                                   embedding=embeddings,
                                   )
retriever = vectorstore.as_retriever()

C:\Users\KDT2\AppData\Local\Temp\ipykernel_49228\777834989.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# Claude LLM
llm = ChatAnthropic(model="claude-opus-4-20250514",
                    temperature=0,
                    max_tokens=1024,
                    api_key=ANTHROPIC_API_KEY,
                    )

# RAG 프롬프트
prompt = hub.pull("rlm/rag-prompt")

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문
question = "분류번호C 외에 분류번호D 질환도 암보험에서 보장되나요?"

# 1. RAG 기반 응답
rag_answer = rag_chain.invoke(question)

# 2. 순수 LLM 응답 (문서 없이 바로 Claude에게 질문)
llm_only_answer = llm.invoke(question)

# 출력
print("🧾 질문:", question)
print("\n💬 [LLM 단독 응답]:")
print(llm_only_answer)
print("\n📚 [RAG 기반 응답]:")
print(rag_answer)



C:\Users\KDT2\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************L20A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}